In [243]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

In [2]:
train = pd.read_csv("./data/train/DataTrain.csv",
                    delimiter=";").drop(columns=["Unnamed: 0"])

train.head()

,Vehicleregistrationplate,NameofFile
0,A7814,DataTrain1.png
1,B1074QO,DataTrain2.png
2,B1031QO,DataTrain3.png
3,B187EDA,DataTrain4.png
4,B1089VD,DataTrain5.png


In [274]:
def upscale(img):
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    path = "EDSR_x4.pb"
    sr.readModel(path)
    sr.setModel("edsr", 4)

    image = sr.upsample(img)
    image = cv2.resize(image, (img.shape[1], img.shape[0]))
    
    return image
    
def denoise(img):
    img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 5, 5)
    
    return img

def denoise_gray(img):
    img = cv2.fastNlMeansDenoising(img, None, 10, 5, 5)
    
    return img

def brightNcontrast(img):
    img = cv2.convertScaleAbs(img, alpha=2, beta=1.5)
    
    return img

def sharpen(img):
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    img = cv2.filter2D(img, -1, kernel)
    
    return img

def to_gray(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    return img

def edge(img):
    se = cv2.getStructuringElement(cv2.MORPH_RECT, (8, 8))
    bg = cv2.morphologyEx(img, cv2.MORPH_DILATE, se)
    img = cv2.divide(img, bg, scale=255)
    img = cv2.threshold(img, 127, 255, cv2.THRESH_OTSU)[1]
    img = cv2.Canny(img, 50, 50)
    
    return img

def preprocess(img):
    img_name = img.split("/")[-1]
    img = cv2.imread(img)
    img = denoise(img)
    img = to_gray(img)
    img = sharpen(img)
    img = denoise_gray(img)
    img = edge(img)
    
    pil_img = Image.fromarray(img)
    pil_img.save(f"./data/processed/{img_name}")
    

def preprocess_test(img):
    img_name = img.split("/")[-1]
    img = cv2.imread(img)
    img = denoise(img)
    img = to_gray(img)
    img = sharpen(img)
    img = denoise_gray(img)
    img = edge(img)

    pil_img = Image.fromarray(img)
    pil_img.save(f"./data/processed_test/{img_name}")

def preprocess_v2(img):
    img_name = img.split("/")[-1]
    img = cv2.imread(img)
    img = denoise(img)
    img = to_gray(img)
    img = sharpen(img)
    img = denoise_gray(img)
    
    pil_img = Image.fromarray(img)
    pil_img.save(f"./data/processed_v2/train/{img_name}")
    
def preprocess_v2_test(img):
    img_name = img.split("/")[-1]
    img = cv2.imread(img)
    img = denoise(img)
    img = to_gray(img)
    img = sharpen(img)
    img = denoise_gray(img)
    
    pil_img = Image.fromarray(img)
    pil_img.save(f"./data/processed_v2/test/tis/{img_name}")

In [261]:
from tqdm import tqdm

In [262]:
img_paths = [f"./data/train/DataTrain{i}.png" for i in range(1, 801)]

for path in tqdm(img_paths):
    preprocess(path)

100%|██████████| 800/800 [00:19<00:00, 40.56it/s]


In [264]:
img_paths_test = [f"./data/test/DataTest{i}.png" for i in range(1, 101)]

for path in tqdm(img_paths_test):
    preprocess_test(path)

100%|██████████| 100/100 [00:04<00:00, 20.44it/s]


In [271]:
img_paths = [f"./data/train/DataTrain{i}.png" for i in range(1, 801)]

for path in tqdm(img_paths):
    preprocess_v2(path)

100%|██████████| 800/800 [00:32<00:00, 24.49it/s]


In [275]:
img_paths_test = [f"./data/test/DataTest{i}.png" for i in range(1, 101)]

for path in tqdm(img_paths_test):
    preprocess_v2_test(path)


100%|██████████| 100/100 [00:03<00:00, 29.37it/s]
